In [1]:
pip install selenium==4.8.2

Note: you may need to restart the kernel to use updated packages.


In [2]:
from bs4 import BeautifulSoup
import selenium
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from webdriver_manager.chrome import ChromeDriverManager
import pandas as pd
import numpy as np
import time
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import re
print(selenium.__version__)

4.8.2


In [3]:
pip install webdriver_manager

Note: you may need to restart the kernel to use updated packages.


In [100]:
import requests

In [145]:
chrome_options = Options()

chrome_options.add_experimental_option("detach", True)
chrome_options.add_experimental_option("excludeSwitches",["enable-logging"])

In [146]:
insert = []
for t in range(1,6) :
    url = 'https://www.musinsa.com/app/codimap/lists?style_type=&tag_no=&brand=&display_cnt=60&list_kind=big&sort=comment_cnt&page={}'.format(t)
    driver = webdriver.Chrome(options=chrome_options)
    driver.get(url)
    for i in range(1,61) :
        new_list = []

        res = requests.get(url, headers={"User-Agent" : "Mozilla/5.0"})
        html = res.text # HTML 텍스트
        soup1 = BeautifulSoup(html, 'html.parser')

        style_list = soup1.find_all('span', class_='style-list-information__text')
        style_append = style_list[i-1].get_text()
        new_list.append(style_append)

        style_title = soup1.find_all('strong', class_='style-list-information__title')
        style_title_append = style_title[i-1].get_text()
        new_list.append(style_title_append)

        post_information = soup1.find_all('span', class_='post-information__text')
        post_append_1 = post_information[3*(i-1)].get_text()
        new_list.append(post_append_1)
        post_append_2 = post_information[3*i-2].get_text().replace('조회', '').strip()
        new_list.append(post_append_2)
        post_append_3 = post_information[3*i-1].get_text().replace('댓글', '').strip()   
        new_list.append(post_append_3)

        driver.find_element(By.XPATH, '/html/body/div[3]/div[2]/form/div[4]/div/ul/li[{}]'.format(i)).click()
        soup = BeautifulSoup(driver.page_source, 'html')
        codimap_explain = soup.find('p', class_='styling_txt').get_text()
        new_list.append(codimap_explain)

        hashtag_found = soup.find_all('a', class_='ui-tag-list__item')    
        codimap_hashtag = []
        for x in range(len(hashtag_found)) :
            codimap_hashtag.append(hashtag_found[x].get_text().strip('#'))
        new_list.append(codimap_hashtag)

        image_url = soup.find('img', class_='photo').get('src')
        new_list.append(image_url)

        insert.append(new_list)
        driver.back()
    driver.close()
    
result_df = pd.DataFrame(insert, columns = ['codimap_category', 'codimap_title', 'codimap_date', 'views', 'comment_numbers', 'codimap_explain', 'codimap_hashtag', 'codimap_imgurl'])

In [154]:
result_df['codimap_hashtag'] = result_df['codimap_hashtag'].apply(lambda x: ', '.join(map(str, x)))

In [155]:
result_df

,codimap_category,codimap_title,codimap_date,views,comment_numbers,codimap_explain,codimap_hashtag,codimap_imgurl
0,캐주얼,톤온톤 코디,21.07.02,"17,269",50,편하게 입기 좋은 반팔 티셔츠와 와이드 팬츠를 연출하고 크로스 백으로 마무리한 캐주얼 룩,"톤온톤, 뉴트럴톤, 캠퍼스, 피크닉, 와이드핏, 오버핏, 꾸안꾸, 여름, 캐주얼, ...",//image.msscdn.net/images/codimap/detail/4035/...
1,캐주얼,블랙 매니아,21.02.23,"36,326",47,편안한 무드의 조거 팬츠에 블랙 스웨트셔츠를 매치한 후 볼캡으로 마무리한 캐주얼 룩,"봄, 캐주얼, 라퍼지스토어, 다이아몬드 레이라, 나이키, 엠프렌즈, 모티패스트, 이...",//image.msscdn.net/images/codimap/detail/1969/...
2,캐주얼,오늘의 코디,22.03.22,"13,400",44,로고 디테일이 돋보이는 스웨트셔츠와 데님 팬츠를 코디하고 스니커즈로 완성한 캐주얼 룩,"로고플레이, 오버핏, 스티치, 캠퍼스, 봄, 캐주얼, 유니폼브릿지, 아디다스, 47...",//image.msscdn.net/images/codimap/detail/13814...
3,댄디,추천 댄디 룩,21.09.15,"21,595",32,담백한 디테일의 니트와 셔츠를 레이어드하고 데님 팬츠를 더해 완성한 댄디 룩,"데이트, 그래픽, 레이어드룩, 가을, 댄디, 엘무드, 뮤지엄바이비컨, 테이크이지, ...",//image.msscdn.net/images/codimap/detail/6746/...
4,캐주얼,릴랙스 룩,21.02.23,"24,027",25,플라워 자수가 눈에 띄는 니트에 조거 팬츠를 더한 후 레드 볼캡으로 컬러 포인트를 ...,"봄, 캐주얼, 카시오, 코닥, 제로, 와릿이즌, 뉴발란스, 커버낫, 디지털, 미니스...",//image.msscdn.net/images/codimap/detail/1974/...
...,...,...,...,...,...,...,...,...
295,댄디,데일리 댄디 룩,21.05.25,"3,555",3,네이비 컬러 반팔 셔츠에 코튼 팬츠를 더하고 스니커즈로 마무리한 댄디 룩,"데이트, 미니멀, 봄, 댄디, 카시오, 아디다스, 로파이, 에잇세컨즈, 쿼츠 아날로...",//image.msscdn.net/images/codimap/detail/3280/...
296,스트릿,유니크하게,22.07.27,"1,942",3,그래픽이 매력적인 반소매 티셔츠와 데님 팬츠를 코디하고 크로스 백으로 연출한 스트릿 룩,"그래픽, 워싱, 와이드핏, 체인, 유니크, 힙합, 여름, 스트릿, 돈애스크마이플랜,...",//image.msscdn.net/images/codimap/detail/17864...
297,스트릿,원 마일 웨어에 빠져,20.10.27,"8,848",3,스포티한 매력의 조거 팬츠에 로고 후드 티셔츠를 매치한 후 브라운 비니로 포인트를 ...,"가을, 스트릿, 칼하트, 제멋, 기프트오리지널, 비니, 셔츠/블라우스, 반소매 티셔...",//image.msscdn.net/images/codimap/detail/245/d...
298,아메카지,벌룬 핏의 매력,22.10.12,"3,635",3,아웃 포켓 디테일이 매력적인 헌팅 재킷과 벌룬 핏 팬츠를 코디하고 스니커즈로 마무리...,"아메카지, 뉴트럴톤, 벌룬핏, 헨리넥, 가을, 아메리칸 캐주얼, 파이살론, 리프로덕...",//image.msscdn.net/images/codimap/detail/19163...


In [156]:
result_df.to_csv("24기_김종진_crawling_추가과제.csv", encoding="euc-kr")